## Workflow overview

The figure below summarizes the end-to-end workflow from cell-based perturbations to phenotype modeling. After measuring multimodal phenotypic readouts and organizing them in an AnnData object, UniPert generates unified perturbagen embeddings across genetic and chemical modalities. These embeddings can be used together with unperturbed profiles as inputs to downstream perturbation-effect predictors.

<img src="https://github.com/user-attachments/assets/bfc669c1-4d41-479b-9da3-f345a18623ef" alt="Workflow overview: UniPert representations in AnnData" width="850">

## Generating UniPert representations for perturb anndata

In this tutorial, we will introduce how to generate perturbagen embeddings using UniPert for a given perturbation `AnnData` file:

* The `UniPert representations` will be formatted as a `dict` and stored in the AnnData object under the key `adata.uns['UniPert_reps']`.
  
* The `invalid or unretrieved perturbagens` will be formatted as a `list` and stored in the AnnData object under the key `adata.uns['invalid_ptbgs']`.
  
We use 2 example perturbation adata from [scPerturb database](https://www.sanderlab.org/scPerturb/datavzrd/scPerturb_vzrd_v2/dataset_info/index_1.html) to show the generating process:

  1. [Example 1: Genetic Perturbation Adata](#Genetic-Perturbation-Adata)

  2. [Example 2: Chemical Perturbation Adata](#Chemical-Perturbation-Adata)




## Prepare example pert adata

Define the download function to get perturbation adata file from [scPerturb database](https://www.sanderlab.org/scPerturb/datavzrd/scPerturb_vzrd_v2/dataset_info/index_1.html).

In [1]:
import os
import requests
from lamin_utils import logger

def download_file(url: str, folder_path: str):
    """
    Download file from url to folder_path
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_name = url.split('/')[-1]
    file_path = os.path.join(folder_path, file_name)
    # check if file already exists
    if os.path.exists(file_path):
        logger.info(f"{file_name} already exists.")
        return file_path
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        logger.download(f"{file_name} download.")
        return file_path
    except requests.exceptions.RequestException as e:
        logger.error(f"Download failed: {e}")
        return None

## Prepare UniPert model

In [3]:
import sys
sys.path.append('../')
from unipert import UniPert

unipert = UniPert()

💡 CUDA is available. Using CUDA.
💡 Constructing UniPert model...
✅ ESM2 model loaded.
✅ Reference ESM2 embedding file loaded.
✅ ESM2 embedder created.
✅ ECFP4 embedder created.
✅ UniPert model constructed.
✅ Pretrained model file loaded.
✅ Reference target graph prepared.
✅ Model loaded and initialized.


## Examples

### Genetic Perturbation Adata

In [4]:
scperturb_url = 'https://zenodo.org/record/10044268/files/PapalexiSatija2021_eccite_arrayed_RNA.h5ad'
demo_data_path = '../demo_data/'  
file_path = download_file(scperturb_url, demo_data_path)

✅ PapalexiSatija2021_eccite_arrayed_RNA.h5ad download.


In [5]:
import scanpy as sc
adata = sc.read(file_path, 'r')
adata

AnnData object with n_obs × n_vars = 8984 × 16826 backed at '../demo_data/PapalexiSatija2021_eccite_arrayed_RNA.h5ad'
    obs: 'perturbation', 'hto', 'guide_id', 'hto_barcode', 'gdo_barcode', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [6]:
adata.obs['perturbation'].value_counts()

perturbation
control     2009
ETV7        1789
IRF1         994
ATF2         794
IRF7         750
MARCH8       723
IFNGR1       701
STAT2        576
CAV1         409
PDL1         235
IFNGR2         4
CMTM6          0
CD86           0
CUL3           0
BRD4           0
PDCD1LG2       0
POU2F2         0
NFKBIA         0
JAK2           0
SPI1           0
SMAD4          0
STAT3          0
STAT1          0
STAT5A         0
TNFRSF14       0
UBE2L6         0
eGFP           0
Name: count, dtype: int64

In [7]:
unipert.enc_ptbgs_for_pert_adata(
    adata=adata,
    ptbg_cols=['perturbation'],
    ptbg_types=['genetic'],
    control_key='control',
    return_results=False
)

💡 Retrieving sequence for genetic perturbagens...


100%|██████████| 10/10 [00:08<00:00,  1.12it/s]

💡 Constructing reference-custom target graph from /data0/lsn1/lisn/VCC/UniPert/data/custom_target_seq.fasta...
💡 Preparing MMseqs and creating reference database...
[04:08:00 +08:00] [mmseqs] [---I---] [thread 2865455] Converting sequences
✅ MMseqs reference database created.
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] 
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for merging to ref_h: 0h 0m 0s 0ms
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for merging to ref: 0h 0m 0s 0ms
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Database type: Aminoacid
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for processing: 0h 0m 0s 87ms
💡 Calculating similarity between /data0/lsn1/lisn/VCC/UniPert/data/custom_target_seq.fasta and reference fasta file...


[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Temporary path /data0/lsn1/lisn/VCC/UniPert/mmseqs_storage/workdir/tmp_ygblIkaz1mDOiqncORkH04MYN does not exist or is not a directory. It will be created.
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Created temporary directory /data0/lsn1/lisn/VCC/UniPert/mmseqs_storage/workdir/tmp_ygblIkaz1mDOiqncORkH04MYN
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Converting sequences
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] 
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for merging to query_h: 0h 0m 0s 0ms
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for merging to query: 0h 0m 0s 0ms
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Database type: Aminoacid
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Time for processing: 0h 0m 0s 0ms
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Call search (subcall): search
[04:08:01 +08:00] [mmseqs] [---I---] [thread 2865455] Se

100%|██████████| 10/10 [00:00<00:00, 12.59it/s]

✅ ESM2 embeddings with 20 querys generated.


✅ UniPert representations generated!
💡 10 perturbagens' UniPert representations saved to adata.uns['UniPert_reps']


In [8]:
adata.uns['UniPert_reps'].keys(), adata.uns['invalid_ptbgs']

(dict_keys(['IRF1', 'STAT2', 'IRF7', 'MARCH8', 'ETV7', 'IFNGR2', 'PDL1', 'IFNGR1', 'ATF2', 'CAV1']),
 [])

### Chemical Perturbation Adata

In [3]:
scperturb_url = 'https://zenodo.org/record/10044268/files/SrivatsanTrapnell2020_sciplex3.h5ad'
demo_data_path = '../demo_data/'  
file_name = download_file(scperturb_url, demo_data_path)

💡 SrivatsanTrapnell2020_sciplex3.h5ad already exists.


In [4]:
import scanpy as sc

adata = sc.read(file_name, 'r')
adata

AnnData object with n_obs × n_vars = 799317 × 110984 backed at '../demo_data/SrivatsanTrapnell2020_sciplex3.h5ad'
    obs: 'ncounts', 'well', 'plate', 'cell_line', 'replicate', 'time', 'dose_value', 'pathway_level_1', 'pathway_level_2', 'perturbation', 'target', 'pathway', 'dose_unit', 'celltype', 'disease', 'cancer', 'tissue_type', 'organism', 'perturbation_type', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'chembl-ID'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [5]:
adata.obs['perturbation'].value_counts()

perturbation
control                              17578
Ellagic acid                          6257
Divalproex Sodium                     6203
Ruxolitinib (INCB018424)              6143
MC1568                                6126
                                     ...  
Alvespimycin (17-DMAG) HCl            2089
Patupilone (EPO906, Epothilone B)     1822
Flavopiridol HCl                      1729
Epothilone A                          1426
YM155 (Sepantronium Bromide)          1007
Name: count, Length: 189, dtype: int64

In [6]:
unipert.enc_ptbgs_for_pert_adata(
    adata=adata,
    ptbg_cols=['perturbation'],
    ptbg_types=['chemical'],
    control_key='control',
    return_results=False
)

✅ chemspider server connected successfully.
💡 Retrievaling SMILES for chemical perturbagens...


  2%|▏         | 4/188 [00:12<09:14,  3.01s/it]

Unable to retrieve SMILES for query compound name: Fedratinib (SAR302503, TG101348)


  3%|▎         | 6/188 [00:17<07:52,  2.59s/it]

Unable to retrieve SMILES for query compound name: Disulfiram 


...

Unable to retrieve SMILES for query compound name: Clevudine 


 99%|█████████▉| 186/188 [08:31<00:05,  2.69s/it]

Unable to retrieve SMILES for query compound name: Regorafenib (BAY 73-4506)


100%|██████████| 158/158 [00:00<00:00, 473.44it/s]


✅ UniPert representations generated!
💡 158 perturbagens' UniPert representations saved to adata.uns['UniPert_reps']
❗ 30 perturbagens can not be repersentated and saved to adata.uns['invalid_ptbgs']: 
['Fedratinib (SAR302503, TG101348)', 'Disulfiram ', 'Busulfan ', 'INO-1001 (3-Aminobenzamide)', 'Glesatinib?(MGCD265)', 'Aminoglutethimide ', 'Prednisone ', 'Cimetidine ', 'ENMD-2076 L-(+)-Tartaric acid ', 'Tazemetostat (EPZ-6438)', 'Mocetinostat (MGCD0103)', 'PD173074', 'Valproic acid sodium salt (Sodium valproate)', 'Streptozotocin (STZ)', 'Luminespib (AUY-922, NVP-AUY922)', 'Cediranib (AZD2171)', 'Lomustine ', 'Nilotinib (AMN-107)', 'Iniparib (BSI-201)', 'Mesna ', 'Azacitidine ', 'Capecitabine ', 'Tacedinaline (CI994)', 'Rucaparib (AG-014699,PF-01367338) phosphate', 'Fluorouracil (5-Fluoracil, 5-FU)', 'Fasudil (HA-1077) HCl', 'Lenalidomide (CC-5013)', 'Bisindolylmaleimide IX (Ro 31-8220 Mesylate)', 'Clevudine ', 'Regorafenib (BAY 73-4506)']


In [7]:
# Try again to retrieve perturbagens not successfully retrieved before
unipert.enc_ptbgs_for_pert_adata(
    adata=adata,
    ptbg_cols=['perturbation'],
    ptbg_types=['chemical'],
    control_key='control',
    return_results=False
)

💡 Retrievaling SMILES for chemical perturbagens...


  0%|          | 0/188 [00:00<?, ?it/s]

Unable to retrieve SMILES for query compound name: Fedratinib (SAR302503, TG101348)


  3%|▎         | 5/188 [00:01<00:56,  3.24it/s]

Unable to retrieve SMILES for query compound name: Disulfiram 


...

Unable to retrieve SMILES for query compound name: Clevudine 


 98%|█████████▊| 184/188 [00:44<00:01,  3.82it/s]

Unable to retrieve SMILES for query compound name: Regorafenib (BAY 73-4506)


100%|██████████| 188/188 [00:46<00:00,  4.05it/s]

✅ UniPert representations generated!
💡 158 perturbagens' UniPert representations saved to adata.uns['UniPert_reps']
❗ 30 perturbagens can not be repersentated and saved to adata.uns['invalid_ptbgs']: 
['Fedratinib (SAR302503, TG101348)', 'Disulfiram ', 'Busulfan ', 'INO-1001 (3-Aminobenzamide)', 'Glesatinib?(MGCD265)', 'Aminoglutethimide ', 'Prednisone ', 'Cimetidine ', 'ENMD-2076 L-(+)-Tartaric acid ', 'Tazemetostat (EPZ-6438)', 'Mocetinostat (MGCD0103)', 'PD173074', 'Valproic acid sodium salt (Sodium valproate)', 'Streptozotocin (STZ)', 'Luminespib (AUY-922, NVP-AUY922)', 'Cediranib (AZD2171)', 'Lomustine ', 'Nilotinib (AMN-107)', 'Iniparib (BSI-201)', 'Mesna ', 'Azacitidine ', 'Capecitabine ', 'Tacedinaline (CI994)', 'Rucaparib (AG-014699,PF-01367338) phosphate', 'Fluorouracil (5-Fluoracil, 5-FU)', 'Fasudil (HA-1077) HCl', 'Lenalidomide (CC-5013)', 'Bisindolylmaleimide IX (Ro 31-8220 Mesylate)', 'Clevudine ', 'Regorafenib (BAY 73-4506)']


In [8]:
len(adata.uns['UniPert_reps'])

158

In [9]:
adata.uns['invalid_ptbgs']

['Fedratinib (SAR302503, TG101348)',
 'Disulfiram ',
 'Busulfan ',
 'INO-1001 (3-Aminobenzamide)',
 'Glesatinib?(MGCD265)',
 'Aminoglutethimide ',
 'Prednisone ',
 'Cimetidine ',
 'ENMD-2076 L-(+)-Tartaric acid ',
 'Tazemetostat (EPZ-6438)',
 'Mocetinostat (MGCD0103)',
 'PD173074',
 'Valproic acid sodium salt (Sodium valproate)',
 'Streptozotocin (STZ)',
 'Luminespib (AUY-922, NVP-AUY922)',
 'Cediranib (AZD2171)',
 'Lomustine ',
 'Nilotinib (AMN-107)',
 'Iniparib (BSI-201)',
 'Mesna ',
 'Azacitidine ',
 'Capecitabine ',
 'Tacedinaline (CI994)',
 'Rucaparib (AG-014699,PF-01367338) phosphate',
 'Fluorouracil (5-Fluoracil, 5-FU)',
 'Fasudil (HA-1077) HCl',
 'Lenalidomide (CC-5013)',
 'Bisindolylmaleimide IX (Ro 31-8220 Mesylate)',
 'Clevudine ',
 'Regorafenib (BAY 73-4506)']